## 瀏覽器&套件設定

In [7]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import traceback
from datetime import datetime


In [8]:
# 擋掉通知黑屏，以防止瀏覽器第一次登入時跳出通知視窗
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})

## 自動登入

In [9]:
# ------ 登入的帳號與密碼 ------
username = "xiao901202@gmail.com"
password="qwerasdf520"

chrome_install = ChromeDriverManager().install()
folder = os.path.dirname(chrome_install)
chromedriver_path = os.path.join(folder, "chromedriver.exe")
service = ChromeService(chromedriver_path)

driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.facebook.com'  
driver.get(url)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))

elem = driver.find_element(By.ID,"email")
elem.send_keys(username)

elem = driver.find_element(By.ID,"pass")
elem.send_keys(password)


# 提交登入
elem.send_keys(Keys.RETURN)
time.sleep(5)

# 檢查有沒有被擋下來
if len(driver.find_elements(By.XPATH,"//*[contains(text(), '你的帳號暫時被鎖住')]")) > 0:
    driver.find_elements(By.XPATH,"//*[contains(text(), '是')]")[1].click()
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), '用戶名稱或密碼無效')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), 'Invalid username or password')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
else:
    feed = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]'))) # 等待動態頁面(鎖定貼文)加載完成
    print("登入成功")

登入成功


## 純貼文爬取

In [18]:

posts_number = int(input("請輸入要爬取的貼文數量:"))
posts = []

driver.set_window_size(1000,4000)
# 初始設定
last_height = driver.execute_script("return document.body.scrollHeight") # 取得頁面高度初始值
last_count = 0 # 計算目前貼文數量

feed = driver.find_element(By.CSS_SELECTOR, 'div[role="main"]') # 找到動態頁面(鎖定貼文)

In [19]:
while True:
    # 如果已經找到足夠的貼文，則停止
    if len(posts) >= posts_number:
        posts = posts[:posts_number]
        break
    
    # 貼文文字的 class selector
    post_text_class_selector = 'div[data-ad-preview="message"]'
    
    # 捕捉所有貼文元素
    
    elements = feed.find_elements(By.CSS_SELECTOR, 'div[class = x1lliihq]')
    current_count = len(elements)    
    if current_count == 0:
        elements = feed.find_elements(By.CSS_SELECTOR, '[data-pagelet]')
        current_count = len(elements)
    

    if current_count != last_count:
        try:
            print("!!! 檢查當前是否有 「查看更多」 Button !!!")
            buttons = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), '查看更多') and @role='button']")))
            for button in buttons:

                # 滾動到按鈕位置
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                WebDriverWait(driver, 5).until(EC.visibility_of(button))
                # 使用 JavaScript 強制點擊按鈕
                driver.execute_script("arguments[0].click();", button)
                print('Button clicked!')
        except Exception as e:
            print(f"當前無「查看更多」 Button")
            pass

        print('!!! 檢查完畢 !!!')
        while len(posts) < posts_number and len(elements) > last_count:
            current_element = elements[last_count]
            last_count += 1
            try:
                print('--------------貼文分隔線--------------')
                # 貼文時間
                try:
                    time_hover = current_element.find_element(By.CSS_SELECTOR, ".x1rg5ohu.x6ikm8r.x10wlt62.x16dsc37.xt0b8zv")
                    time_info = ActionChains(driver).move_to_element(time_hover)
                    time_info.perform()
                    time_tooltip = WebDriverWait(driver, 6).until(
                            EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[role="tooltip"]'))
                    )
                    tooltip_text = time_tooltip.text
                    print(tooltip_text)
                except:
                    print('無法取得貼文時間')
                    tooltip_text = ''
                
                # 貼文作者
                try:
                    author = current_element.find_element(By.CSS_SELECTOR, 'h4')
                    author_text = author.text.split('\n')[0]
                    print(author_text)
                except:
                    print('無法取得作者名稱')
                    author_text = ''
                # 貼文作者資訊(類型、產業)
                try:
                    # print(current_element.get_attribute('innerHTML'))
                    print('觸發作者攔')
                    author_info_hover = current_element.find_element(By.CSS_SELECTOR, '.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1sur9pj.xkrqix3.xzsf02u.x1s688f')
                    # author_info_hover = current_element.find_element(By.CSS_SELECTOR, '.html-h4.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1vvkbs.x1heor9g.x1qlqyl8.x1pd3egz.x1a2a7pz.x1gslohp.x1yc453h')
                    author_info = ActionChains(driver).move_to_element(author_info_hover)
                    print(author_info_hover.text)
                    author_info.perform()
                    print(driver.get_attribute('innerHTML'))

                    author_info_modal = WebDriverWait(driver, 5).until(
                        EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[aria-label="連結預覽"]'))
                    )
                    if '·' in author_info_modal.text:
                        author_info_text = author_info_modal.text.split("\n")
                        author_type = author_info_text[1].split(" · ")[0]
                        industry = author_info_text[1].split(" · ")[1]
                        print(author_type)
                        print(industry)
                    else:
                        author_type = author_info_modal.text
                        print(author_type)
                        industry = ''
                except Exception as e:
                    traceback.print_exc()  
                    print('無法取得作者資訊')
                    print(e)
                    author_type = ''
                    industry = ''

                # 作者追蹤者數
                follower = ''
                for i in author_info_text:
                    if '位追蹤者' in i:
                        follower = i[:-4]
                        break
                print(follower)

                # 貼文按讚數
                try:
                    like_count = current_element.find_element(By.CSS_SELECTOR, '.xrbpyxo.x6ikm8r.x10wlt62.xlyipyv.x1exxlbk')
                    like_count_text = like_count.text
                    print(like_count_text)
                except:
                    print('無法取得按讚數')
                    like_count_text = ''
                

                # 貼文留言與分享數
                try:
                    comment_share = current_element.find_element(By.CSS_SELECTOR, '.x9f619.x1n2onr6.x1ja2u2z.x78zum5.x2lah0s.x1qughib.x1qjc9v5.xozqiw3.x1q0g3np.xykv574.xbmpl8g.x4cne27.xifccgj')
                    comment_share_text = comment_share.text
                    comment_count = comment_share_text.split('\n')[0]
                    share_count = comment_share_text.split('\n')[1]

                    print('貼文留言與分享數')
                    print(comment_count, share_count)
                except Exception as e:
                    print(e)
                    comment_count = ''
                    share_count = ''
                    print('無法取得留言與分享數')                    
                    
                # 貼文內容
                element_post = current_element.find_element(By.CSS_SELECTOR, post_text_class_selector)
                post_text = element_post.text
                posts.append({'author': author_text,'author_type':author_type,'industry':industry,'follower':follower ,'time':tooltip_text, 'text': post_text ,'like_count':like_count_text,'comment_count':comment_count,'share_count':share_count})
                
                print(post_text)
            except Exception as e:
                print(e)
                print('此篇非普通貼文，跳過處理')
        print('---------------------------------')

    # 計算新的滾動高度並比較是否到達底部
    new_height = driver.execute_script("return document.body.scrollHeight")
    last_height = new_height

    # 滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等待新內容加載(秒)
    time.sleep(3)
# driver.refresh()
print('共爬取', len(posts), '篇貼文')
print(posts)

!!! 檢查當前是否有 「查看更多」 Button !!!
Button clicked!
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
2024年9月20日 星期五上午10:58
電腦零件便宜拍賣
觸發作者攔
電腦零件便宜拍賣


Traceback (most recent call last):
  File "C:\Users\BL515\AppData\Local\Temp\ipykernel_6772\3586256639.py", line 72, in <module>
    author_info_modal = WebDriverWait(driver, 5).until(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BL515\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x0

無法取得作者資訊
Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x00A63DD8]
	(No symbol) [0x00A63F75]
	(No symbol) [0x00A56406]
	BaseThreadInitThunk [0x768B7BA9+25]
	RtlInitializeExceptionChain [0x77C3C11B+107]
	RtlClearBits [0x77C3C09F+191]

5,194 
無法取得按讚數
list index out of range
無法取得留言與分享數
華碩gtx750ti 600 元
顯卡之前退下 良品
伺服器DDR4 3200 16G 400元
DDR3 16008Gx4一隻200元
DDR4 2400T 8g一隻250元
WD Ssd 250G 300元
I3 6100 200元
E3-1230v2 250元
（伺服器處理器E-2224  3000元）
全收另算
沒賣便貨 沒賣便貨
個保三天,三天內有問題可退
自取佳 來時做好防疫措施謝謝
---------------------------------
!!! 檢查當前是否

Traceback (most recent call last):
  File "C:\Users\BL515\AppData\Local\Temp\ipykernel_6772\3586256639.py", line 72, in <module>
    author_info_modal = WebDriverWait(driver, 5).until(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BL515\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x0

無法取得作者資訊
Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x00A63DD8]
	(No symbol) [0x00A63F75]
	(No symbol) [0x00A56406]
	BaseThreadInitThunk [0x768B7BA9+25]
	RtlInitializeExceptionChain [0x77C3C11B+107]
	RtlClearBits [0x77C3C09F+191]

5,194 
26
貼文留言與分享數
1 則留言 3次分享
絕沒有「穩賺」、「暴賺」
只有在大漲大跌的過程告訴您「怎麼辦」
「復華全明星隊」有效運作近 25 年
 給你輕鬆一鍵下單的台股《母子基金》組合
→ https://pse.is/6dd8sq
母基金(穩健型)：
復華台灣智能基金
子基金(積極型)：
① 復華全方位基金
② 復華復華基金
 復華投信相信
「長期」和「紀律」是投資致勝不變的原則
 2000 年領先同業
推出金複合(母子基金)投資法
秉持投資致勝原則
陪伴客戶歷經大漲、暴跌，累積多年經驗值
破解「何時買」、「何時賣」的投資難題
#四分之一世紀的傳承 
#復華投信

...

無法取得作者資訊
Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x00A63DD8]
	(No symbol) [0x00A63F75]
	(No symbol) [0x00A56406]
	BaseThreadInitThunk [0x768B7BA9+25]
	RtlInitializeExceptionChain [0x77C3C11B+107]
	RtlClearBits [0x77C3C09F+191]

5,194 
323
貼文留言與分享數
11則留言 53次分享
以未成年懷孕為主題，清新溫暖的寫實系作品。
兩個16歲孩子在偷嚐禁果後意外懷孕了。面對突然到來的新生命，小福跟阿寶細細思考、反覆討論，然後勇敢的決定！對於沒有唯一正解的問題，能做的只有選擇當下的心之所向，並且努力讓周遭的人了解與接受。
這部作品不灑狗血、沒有老梗，取而代之的是許多充滿溫度的場景，好多場溫柔真摯的對話都讓人眼眶泛淚，最終呈現了一個有著美好未來的結局，或許真實世界可能會再多點殘酷，但這個故事仍讓觀眾充滿共鳴與感動。真的希望所有的孩子，都能在這樣充滿善意的世界成長，犯了錯、跌了跤、出了事，總是會有人接住。
本

Traceback (most recent call last):
  File "C:\Users\BL515\AppData\Local\Temp\ipykernel_6772\3586256639.py", line 72, in <module>
    author_info_modal = WebDriverWait(driver, 5).until(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BL515\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x0

無法取得作者資訊
Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x00A63DD8]
	(No symbol) [0x00A63F75]
	(No symbol) [0x00A56406]
	BaseThreadInitThunk [0x768B7BA9+25]
	RtlInitializeExceptionChain [0x77C3C11B+107]
	RtlClearBits [0x77C3C09F+191]

5,194 
1,509
貼文留言與分享數
81則留言 163次分享
假設你是男主的話你會想和誰結婚
---------------------------------
!!! 檢查當前是否有 「查看更多」 Button !!!
Button clicked!
Button clicked!
Button clicked!
Button clicked!
Button clicked!
Button clicked!
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
無法取得貼文時間
台灣角川
觸發作者攔
台灣角川
  · 追蹤


Traceback (most recent call last):
  File "C:\Users\BL515\AppData\Local\Temp\ipykernel_6772\3586256639.py", line 72, in <module>
    author_info_modal = WebDriverWait(driver, 5).until(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BL515\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x0

無法取得作者資訊
Message: 
Stacktrace:
	GetHandleVerifier [0x00ACD933+25811]
	(No symbol) [0x00A5E314]
	(No symbol) [0x00952523]
	(No symbol) [0x00997483]
	(No symbol) [0x0099765B]
	(No symbol) [0x009D6BA2]
	(No symbol) [0x009BAD74]
	(No symbol) [0x009D46F0]
	(No symbol) [0x009BAAC6]
	(No symbol) [0x0098BEFD]
	(No symbol) [0x0098C8FD]
	GetHandleVerifier [0x00D9F143+2981091]
	GetHandleVerifier [0x00DF2FF9+3324825]
	GetHandleVerifier [0x00B5B32F+605903]
	GetHandleVerifier [0x00B62CBC+637020]
	(No symbol) [0x00A66F4D]
	(No symbol) [0x00A63DD8]
	(No symbol) [0x00A63F75]
	(No symbol) [0x00A56406]
	BaseThreadInitThunk [0x768B7BA9+25]
	RtlInitializeExceptionChain [0x77C3C11B+107]
	RtlClearBits [0x77C3C09F+191]

5,194 
2,445
貼文留言與分享數
44則留言 58次分享
有希：「未免太粗心大意了吧？」
TV動畫《不時輕聲地以俄語遮羞的鄰座艾莉同學》動畫第11集改編自小說第3集，開頭探訪政近的艾莉其實就是第3集封面，而將政近支開後貼近艾莉的有希終於露出狡猾的一面。與小說懾人笑容不同的是，動畫採用較為冷面笑匠的形式，給人一種最終BOSS感。
想了解原作是如何描繪的話，現在去追小說第3集還來得及趕上動畫最終回哦！
#ロシデレ #遮羞艾莉 #新番動畫 #台灣角川 #原作小說推薦
---------------------------------
共爬取 5 篇貼文
[{'author': 

In [17]:
driver.refresh()

In [6]:
import pandas as pd
now = datetime.now()
formatted_date = now.strftime("%Y%m%d_%H%M%S_posts")

df = pd.DataFrame(posts)
df.to_csv(f'{formatted_date}.csv', index=False, encoding='utf_8_sig')
# 显示 DataFrame
print(df)

                                   author author_type industry follower  \
0                               2000Fun遊戲        粉絲專頁   電子商務網站     16 萬   
1                             闈鴻 WEI HONG        粉絲專頁    影片創作者   5,194    
2                                   OW看動漫                        5,194    
3                                   自由時報                         5,194    
4  Kameyou 加美尤 Blog 在【我推的孩子】相簿中新增了 1 張相片。                        5,194    

                    time                                               text  \
0  2024年9月18日 星期三上午10:56  中秋節翌日假期（9月18日） – 2000Fun特別營業時間通知：\n2000Fun全線門市...   
1                         好羨慕台灣人的鹹水雞皮膚\n 毛髮很少，沒有我這個中東傢伙掉毛的煩惱\n。但住了台灣三年的我...   
2   2024年9月14日 星期六上午7:00  【被班上的同學催眠了....嗎?】\n作者:\n御家かえる\n@GoHome_kun\n(這...   
3   2024年9月20日 星期五上午7:18  見證歷史的時刻！無人可達的境界！\n賽前還差兩轟一盜可達成史上第一人50轟50盜，結果大谷告...   
4   2024年9月19日 星期四上午1:25  【命運的擁抱】\n１、阿奎亞打算分手之際，心底竟想起了黑川茜的擁抱——原來自己眷戀着茜的溫柔...   

  like_count comment_count share_count  
0          3                     

In [15]:
a = 'html-h4 x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1vvkbs x1heor9g x1qlqyl8 x1pd3egz x1a2a7pz x1gslohp x1yc453h'
a = a.replace(' ', '.')
a

'html-h4.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1vvkbs.x1heor9g.x1qlqyl8.x1pd3egz.x1a2a7pz.x1gslohp.x1yc453h'